Customer Purchases Analysis:


In [17]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r"C:\Users\Asus\Downloads\chinook.db")


customers = pd.read_sql("SELECT * FROM customers", conn)
invoices = pd.read_sql("SELECT * FROM Invoices", conn)
invoice_items = pd.read_sql("SELECT * FROM Invoice_items", conn)
tracks = pd.read_sql("SELECT TrackId, AlbumId FROM tracks", conn)
albums = pd.read_sql("SELECT * FROM playlists", conn)

invoice_details = pd.merge(invoice_items,invoices, on='InvoiceId')
total_per_customer = invoice_details.groupby('CustomerId')['UnitPrice'].sum().reset_index()
total_per_customer = pd.merge(total_per_customer,customers[['CustomerId','FirstName','LastName']],on='CustomerId')

top5 = total_per_customer.sort_values(('UnitPrice'), ascending = False).head(5)
top5 = top5[['CustomerId','FirstName','LastName','UnitPrice']]
top5



,CustomerId,FirstName,LastName,UnitPrice
5,6,Helena,Holý,49.62
25,26,Richard,Cunningham,47.62
56,57,Luis,Rojas,46.62
44,45,Ladislav,Kovács,45.62
45,46,Hugh,O'Reilly,45.62


In [22]:
invoice_data = pd.merge(invoice_items, invoices, on='InvoiceId')

# Step 2: Merge with tracks (to get AlbumId)
invoice_tracks = pd.merge(invoice_data, tracks, on='TrackId')
customer_album_counts = invoice_tracks.groupby(['CustomerId','AlbumId']).size().reset_index(name='TracksBought')
album_track_counts = tracks.groupby('AlbumId').size().reset_index(name='TotalTracks')
merged = pd.merge(customer_album_counts,album_track_counts,on='AlbumId')
merged['PurchaseType'] = merged.apply(
    lambda x: 'Album' if x['TracksBought'] == x['TotalTracks'] else 'Individual', axis =1
)

customer_pref = merged.groupby('CustomerId')['PurchaseType'].apply(
    lambda x: x.mode()[0]
).reset_index()

summary = customer_pref['PurchaseType'].value_counts(normalize=True)*100
summary


PurchaseType
Individual    100.0
Name: proportion, dtype: float64